In [4]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Imports

In [30]:
import os
import cv2
import sys
import tifffile
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from collections import Counter
from matplotlib import pyplot as plt

sys.path.append("../code/")

In [15]:
from data.dataset import load_image
from utils.rle import *
from params import *

### Preparation

In [46]:
FACTOR = 4

In [47]:
out_dir = DATA_PATH + f"train_{FACTOR}/"

In [48]:
df_masks = pd.read_csv(DATA_PATH + "train.csv").set_index("id")

In [49]:
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

### Loop

In [31]:
masks = {}

for index, encs in tqdm(df_masks.iterrows(), total=len(df_masks)):
    # read image and generate the mask
    img = load_image(os.path.join(TIFF_PATH, index + ".tiff"))
    mask = enc2mask(encs, (img.shape[1], img.shape[0]))

    img = cv2.resize(
        img,
        (img.shape[1] // FACTOR, img.shape[0] // FACTOR),
        interpolation=cv2.INTER_AREA,
    )

    mask = cv2.resize(
        mask,
        (mask.shape[1] // FACTOR, mask.shape[0] // FACTOR),
        interpolation=cv2.INTER_NEAREST,
    )
    
    rle = mask2enc(mask)
    
    masks[index] = rle
    
#     tifffile.imsave(out_dir + f"{index}.tiff", img)

In [52]:
df_masks = pd.DataFrame.from_dict(masks).T.reset_index().rename(columns={0: "encoding", "index": "id"})

In [53]:
df_masks

,id,encoding
0,2f6ecfcdf,18509325 4 18509331 6 18517140 20 18524955 28 ...
1,aaa6a05cc,1938681 15 1943301 19 1947920 24 1952540 28 19...
2,cb2d976f4,4887497 1 4896226 11 4904955 21 4913684 31 492...
3,0486052bb,6358179 2 6364624 4 6371069 7 6377513 10 63775...
4,e79de561c,470499 21 474544 23 478588 25 482632 28 486677...
5,095bf7a1f,7092936 6 7102474 35 7112013 39 7121551 44 713...
6,54f2eec69,7795325 37 7802932 41 7810540 44 7818147 49 78...
7,1e2425f28,3097395 8 3104086 17 3110776 26 3117470 30 312...


In [55]:
df_masks.to_csv(f"../input/train_{FACTOR}.csv", index=False)